In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

<br>
<br>
# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.
<br>
<br>
<br>

In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import random
import tensorflow as tf
from tensorflow import keras
from konlpy.tag import Okt
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from string import punctuation
from tensorflow.python.keras.preprocessing import sequence
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from collections import Counter
from keras.utils import np_utils
from tensorflow.python.keras.preprocessing import sequence
# from gensim.models import Word2Vec
okt = Okt()
import re
pattern1 = re.compile(r"[{0}]".format(re.escape(punctuation)))
#pattern2 = re.compile(r"[A-Za-z]{5,}") # 영어 5섯글자 이상 날림
pattern2 = re.compile(r"[^ \nA-Za-z]") ## "" 인용문 ,특수문자 이런거 다날림 최대한 꺠끗한 단어 얻는게 목적
pattern3 = re.compile(r"\s{2,}") ## 공백2개이상인 부분 공백 하나로 

from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\legen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\legen\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [59]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드

# chatbot_data = pd.read_csv('./dataset/chatbot/ChatbotData.csv', encoding='utf-8')
# question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])


temp_text=[]
temp_topic=[]
question = []
answer = []
temp=[]


with open("./dataset/chatbot/dialogues_text.txt","r",encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        temp.append(line.split("\n"))
    file.close()
    


for i in range(0,500):
#     print(temp[i])
#     print("=================================")
    temp2 = temp[i][0].split("__eou__")
#     print(temp2)
#     print(len(temp2))
#     print('\n')

    if (len(temp2)-1 )% 2== 0 :
        for j in range(0,len(temp2)-1):

            if j%2 == 0:
                question.append(temp2[j])
            else:
                answer.append(temp2[j])
    

    
# for i in range(0,500):
#     print(temp[i])
#     print("                  ")
#     print(temp[i].split("__eou__"))
#     print("=====================")
#    # train_question.append()
        
        #train_set.append((int(temp[i][0].split("\t")[1]),temp[i][0].split("\t")[0]))


In [60]:
# temp =[]
# with open("./dataset/chatbot/dialogues_topic.txt","r",encoding='utf-8') as file:
#     lines = file.readlines()
#     for line in lines:
#         temp.append(line.split(" "))

#     file.close()
# print(len(temp))
    

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다. 질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다. 감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.

https://github.com/songys/Chatbot_data
<br>
<br>
<br>

<br>
<br>
# 단어 사전 생성

In [61]:
question[1]

'Are things still going badly with your houseguest ? '

In [62]:
def pos_tag(corpus):
    
    for i in range(0,len(corpus)):
        
        text =   pattern3.sub(" ",pattern2.sub(" ",pattern1.sub("",corpus[i]))).strip()
        no_capitals = text.lower().split(' ')
    
# #     # 불용어 제거
#     stops = set(stopwords.words('english'))
#     no_stops = [word for word in no_capitals if not word in stops if len(word)>1]
    

        stemmer = nltk.stem.SnowballStemmer('english')
        stemmer_words = [stemmer.stem(word) for word in no_capitals]
        
        ttemp=''
        for j in stemmer_words:
            ttemp = ttemp + j + ' '
            
        corpus[i] = ttemp.strip()
        print(corpus[i])
#     for token in stemmer_words :
#         if token in word_index:
#             indexes.append(word_index[token])
#         else:
#             indexes.append(oov_id)
#     print(stemmer)
#     print(stemmer_words)
    return corpus

In [63]:
pos_tag(['we have a lot of chinesses restaurant in america'])

we have a lot of chiness restaur in america


['we have a lot of chiness restaur in america']

In [64]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


the kitchen stink
are thing still go bad with your houseguest
leo i realli think you re beat around the bush with this guy i know he use to be your best friend in colleg but i realli think it s time to lay down the law
would you mind wait a while
im not sure but ill get a tabl readi as fast as i can
can i take your order now or do you still want to look at the menu
certain how about spaghetti with clam and shrimp
im exhaust
what kind of food do you like
but your american
would you like to take a look at the menu sir
would you care for a drink befor you order
yes sir ill bring it over have you decid what youd like sir
can i help you sir what do you need
of cours sir no problem
could i have my bill pleas
im afraid there been a mistak
i believ you have charg me twice for the same thing look the figur of dollar appear here then again here
may i sit here
is everyth to your satisfact
oh sorri to hear that this is quit unusu as we have steak from the market everi day
im sorri sir do you wish 

is there a better tailor around here
what can i offer you here
ok pleas go ahead
oh we are skill in do that you can trust us our shop has a histori of more than year
yes absolut we have a veri good reput in the neighborhood
if you like you can wait here they will be readi in one hour and a half
can i have the menu pleas
could you pleas pass me the salt
have you got chang for a thousand
excus me im a littl lost which bus do i take to get to shi da
oh ok where do i get off
ok thank
no is it difficult
how much doe it cost
wow i cant thank you enough
what can i do for you
when do you need them sir
no problem do you want x or x
and do you want the glossi or mat finish
you pay when you pick them up i dont need a deposit for just one roll of film
do you develop film here
well the x is fine
the last one is black and white all the rest should need color
i hope they will come out well when should i pick them up
yes it is and develop them as glossi as possibl
yes a roll of kodak film pleas
may i 

no i think i will run thing much the same way that dad did
we will go to climb up the hill tomorrow let go to supermarket to buy some food
what should we buy
it is heavi and not conveni to take them to the top
good idea we can buy some candi and nut for children
that enough for the kid get some drink
right we should buy some yoghurt for kid
ok four saucer we can buy some custard tart tomorrow morn
can i help you
what kind of stamp do you want
i must weigh it
it is gram overweight do you want to send it as an ordinari or regist letter
registr plus overweight yuan in all
hi barbara have you lost weight
wow you look great
what type of diet were you on
that amaz what tip did your trainer give you
that doesnt make sens if you eat more than how do you lose weight
that make sens im go to tri that
i have a littl gut i have been tri to get rid of
thank for the advic
nathan said he didnt have ani fun at his new school this week do you know whi
he realli miss his friend from our old neighborhood


after a while good afternoon mrs smith i call you just now
yes
wonder a modest comfort room ah a beauti window through which i can see a beauti scene and hear the bird sing how nice
excel i like it when can i move in
ok
of cours what the problem
sure thing
did i
i take it youv done this befor
good
id rather have a cup of coffe
yes pleas milk and two sugar
pass him
i was worri where you were
yes you should have
yes ive got to work overtim
so what is it
anoth tough thing i guess
ill be back late tonight
i have a meet with my general manag
i d love to but i m a terribl dancer i m afraid i may step on your toe
thank you i ll tri my best
don t worri about it i have his pager number right here
forget it it s noth
it delici
okay say when
no thank im fine
im sorri hes out at the moment
i think hell be back in about an hour at least
yes of cours
ok doe he know your number
all right thank for call oh hang on someon at the door that may be him pleas hold on
it good to be home
take the no bus ther

it not expens ill take it
we have a great varieti of chines antiqu and porcelain do you like them
you are right how about paint and embroid handkerchief
of cours these paint have mani design such as anim flower landscap aquat and so on
all right chines embroideri is one tradit art with elabor design and high qualiti
this is a seri of anim call chines zodiac
ok i will wrap them for you
there some leftov pizza from last night
um i think there may be some leftov lasagna from the other day
there might be some birthday cake left in the freezer
you can wash the veget and cut the chicken and ill make dinner
thank you for notic ive been make a few lifestyl chang over the past year
well it all start when my brother challeng me to quit smoke
when i kick that habit i found i had so much more energi i start exercis
no i like to be outsid i took up hike and mountain bike
how did you final do it
like snack
i get what you mean
im impress good for you
i didnt know you did yoga
what have you learn
as o

im not use to other brand
ok i hope so
good afternoon i want to pick up a new shaver sinc the old one has retir
im still not sure the type with top qualiti i guess well which one do you recommend
it doesnt matter much if it has high qualiti and reason price it will be ok
do you have a warranti on it
if there someth wrong with it in a week can i come back to chang it
ok ill take one
i want to go to the supermarket
let get a shop cart
what are you go to buy
ok id like some orang but they dont look quit ripe yet
ill take some too
ok ill go and buy some cake
no let go over to the checkout stand
ok could you pleas help me to clean them
great at first you can put the chair back
thank you ill wash them
good boy
what did you do
you have a lot of plant
that take some time
ill bet he enjoy his walk
that a lot of work
ill bet you were hungri
yes ive had one for as long as i can rememb
i actual like have a curfew i have an excus to get to bed on time
i dont like be too tire the next day
if you sta

In [65]:
len(ques)

NameError: name 'ques' is not defined

In [ ]:
len(words)

In [ ]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [ ]:
words

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다. 자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다. 그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.
<br>
<br>
<br>

In [ ]:
# 단어 개수
len(words)


In [ ]:
# 단어 출력
words[:20]


In [ ]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [ ]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])


In [66]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: 'stream',
 5: 'tick',
 6: 'ribero',
 7: 'posh',
 8: 'scratch',
 9: 'tome',
 10: 'temperatur',
 11: 'penc',
 12: 'midsiz',
 13: 'rd',
 14: 'shortcak',
 15: 'cinnamon',
 16: 'prioriti',
 17: 'blood',
 18: 'magazin',
 19: 'petra'}

<br>
<br>
# 전처리

In [67]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END

원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. 

그러나 예측시에는 이런 방식이 불가능합니다. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다. 모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.
<br>
<br>
<br>

In [68]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]


array([3629, 6106, 1676,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [69]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


array([   1, 1809, 5874, 3414, 3629, 5059,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [70]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
len(y_decoder)


1098

In [71]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

<br>
<br>
# 모델 생성

In [72]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

지금까지의 예제는 Sequential 방식의 모델이었습니다. 하지만 이번에는 함수형 API 모델을 사용했습니다. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. 

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다. 그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.
<br>
<br>
<br>

In [73]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

<br>
<br>
# 훈련 및 테스트

In [74]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [75]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()


Total Epoch : 1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


KeyboardInterrupt: 

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

<br>
<br>
# 문장 생성

In [ ]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [ ]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 놀러가고 싶다')
input_seq


In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 같이 놀러가고 싶다')
input_seq


In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('강아지 키울까?')
input_seq


In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


하지만 데이터셋에 없던 '가려고'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다. 이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다. 데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.